In [ ]:
# Importing the libraries 
import warnings 
import itertools
'''import itertools: This imports the Python itertools module, which provides various functions for 
working with iterators and iterable objects. In this code, itertools.product is used to generate 
combinations of parameters for the SARIMA model.'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
'''import matplotlib.pyplot as plt: This imports the pyplot module from the Matplotlib library 
and aliases it as plt. Matplotlib is a popular library for creating visualizations in Python.'''

warnings.filterwarnings("ignore")
'''warnings.filterwarnings("ignore"): This line sets the warning filter to ignore all warning messages. 
It prevents warning messages from being printed during the program's execution.'''
